In [1]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys #tuve que usar esto para poder importar seaborn
!{sys.executable} -m pip install seaborn
import seaborn as sns
from sklearn import metrics, ensemble, decomposition
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import display
%matplotlib inline 
np.random.seed(742390)

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.11/libexec/bin/python3 -m pip install --upgrade pip' command.


Preparación y M1

In [2]:
df = pd.read_csv("data.csv")
print(f"N° de datos: {len(df)}")
display(df.head())
display(df.describe())

N° de datos: 234150


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1.0,5.0,NaN,5263,555.0,1.0,1.0,0,1.0
1,NaN,NaN,2015-07-31,6064,625.0,1.0,1.0,NaN,1.0
2,3.0,5.0,2015-07-31,8314,821.0,1.0,1.0,NaN,1.0
3,NaN,5.0,NaN,13995,1498.0,NaN,1.0,NaN,1.0
4,5.0,5.0,2015-07-31,4822,559.0,NaN,1.0,0,1.0


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,189361.000000,201401.000000,234150.000000,195385.000000,177926.000000,177732.000000,177426.000000
mean,558.425642,3.999955,5901.379671,626.977936,0.832408,0.404806,0.141276
std,321.841666,2.000309,3832.485440,452.442907,0.373505,0.490856,0.348307
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,3925.000000,413.000000,1.000000,0.000000,0.000000
50%,559.000000,4.000000,5930.000000,608.000000,1.000000,0.000000,0.000000
75%,837.000000,6.000000,8008.000000,824.000000,1.000000,1.000000,0.000000
max,1115.000000,7.000000,41551.000000,5458.000000,1.000000,1.000000,1.000000


In [3]:
df = df.dropna(thresh=6)
pd.isna(df).value_counts()

Store  DayOfWeek  Date   Sales  Customers  Open   Promo  StateHoliday  SchoolHoliday
False  False      False  False  False      False  False  False         False            25112
                  True   False  False      False  False  False         False            13783
                  False  False  False      False  False  True          False            13059
                                                  True   False         False             8050
                                                  False  False         True              8008
                                                                                        ...  
True   True       False  False  False      False  False  False         True               284
False  True       False  False  True       False  True   False         False              271
                                                  False  False         True               247
                                           True   False  False       

In [4]:
label_encoder = LabelEncoder()
for i in ["DayOfWeek", "SchoolHoliday","Date"]:
    df[i] = label_encoder.fit_transform(df[i])
df["Store"] = label_encoder.fit_transform(df["Store"].astype(str))
df["Sales"] = label_encoder.fit_transform(df["Sales"].astype(str))
df["Customers"] = label_encoder.fit_transform(df["Customers"].astype(str))
df["Open"] = label_encoder.fit_transform(df["Open"].astype(str))
df["Promo"] = label_encoder.fit_transform(df["Promo"].astype(str))
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,0,4,210,11366,2715,1,1,0,1
1,1115,7,209,12170,2785,1,1,NaN,1
2,338,4,209,14443,2986,1,1,NaN,1
4,560,4,209,10925,2719,2,1,0,1
5,671,7,209,11755,2749,1,1,NaN,1


Como se puede observar, tanto para DayOfWeek como para StateHoliday hay algunos datos faltantes. 3 y 2, respectivamente.

In [5]:
cond1 = ~(df["Store"].isna() | df["DayOfWeek"].isna() | df["Date"].isna() | df["Sales"].isna() | df["Customers"]| df["Open"]| df["Promo"]| df["StateHoliday"]| df["SchoolHoliday"].isna())
df_pred = df[cond1].copy()
training_set, test_set = train_test_split(df_pred.copy(), test_size = 0.3)

print(f'Tamaño set entrenamiento: {len(training_set)}')
print(f'Tamaño set test: {len(test_set)}')

Tamaño set entrenamiento: 3972
Tamaño set test: 1703


In [6]:
scaler = StandardScaler()
var_num = ["Sales"]

training_set[var_num] = scaler.fit_transform(training_set[var_num])
test_set[var_num] = scaler.transform(test_set[var_num])

training_set.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
21196,139,6,190,0.0,0,0,0,NaN,0
75362,1115,0,142,0.0,0,0,0,NaN,0
87417,503,3,210,0.0,0,0,0,NaN,0
14407,1115,6,197,0.0,0,0,0,NaN,2
185505,468,6,210,0.0,0,0,0,NaN,0


In [7]:
#features = ["Store", "DayOfWeek", "Date", "Sales","Customers","Open","Promo","StateHoliday","SchoolHoliday"]
#target = "Sales"
#reg1 = LinearRegression()
#reg1.fit(training_set[features], training_set[target])
#predictions = reg1.predict(test_set[features])
#mse = metrics.mean_squared_error(test_set[target], predictions)
#print(f"Error cuadrático medio: {mse}")

In [9]:
df = df.dropna(thresh=6)
pd.isna(df).value_counts()

Store  DayOfWeek  Date   Sales  Customers  Open   Promo  StateHoliday  SchoolHoliday
False  False      False  False  False      False  False  False         False            25112
                  True   False  False      False  False  False         False            13783
                  False  False  False      False  False  True          False            13059
                                                  True   False         False             8050
                                                  False  False         True              8008
                                                                                        ...  
True   True       False  False  False      False  False  False         True               284
False  True       False  False  True       False  True   False         False              271
                                                  False  False         True               247
                                           True   False  False       

In [10]:
values1 = set(df['DayOfWeek']).difference(set(['.', np.nan]))
dow_translator = {}
for i, k in enumerate(values1):
    dow_translator[k] = i+1
    dow_translator[i+1] = i+1

In [11]:
values2 = set(df['Date']).difference(set(['.', np.nan]))
date_translator = {}
for i, k in enumerate(values2):
    date_translator[k] = i+1
    date_translator[i+1] = i+1

In [12]:
values3 = set(df['Sales']).difference(set(['.', np.nan]))
sales_translator = {}
for i, k in enumerate(values3):
    sales_translator[k] = i+1
    sales_translator[i+1] = i+1

In [13]:
values4 = set(df['Customers']).difference(set(['.', np.nan]))
customers_translator = {}
for i, k in enumerate(values4):
    customers_translator[k] = i+1
    customers_translator[i+1] = i+1

In [14]:
values5 = set(df['Open']).difference(set(['.', np.nan]))
open_translator = {}
for i, k in enumerate(values5):
    open_translator[k] = i+1
    open_translator[i+1] = i+1

In [15]:
values6 = set(df['Promo']).difference(set(['.', np.nan]))
promo_translator = {}
for i, k in enumerate(values6):
    promo_translator[k] = i+1
    promo_translator[i+1] = i+1

In [16]:
values7 = set(df['StateHoliday']).difference(set(['.', np.nan]))
stateholiday_translator = {}
for i, k in enumerate(values7):
    stateholiday_translator[k] = i+1
    stateholiday_translator[i+1] = i+1

In [17]:
values8 = set(df['SchoolHoliday']).difference(set(['.', np.nan]))
schoolholiday_translator = {}
for i, k in enumerate(values8):
    schoolholiday_translator[k] = i+1
    schoolholiday_translator[i+1] = i+1

In [18]:
dow_translator[np.nan] = np.nan
dow_translator['.'] = np.nan

date_translator[np.nan] = np.nan
date_translator['.'] = np.nan

sales_translator[np.nan] = np.nan
sales_translator['.'] = np.nan

customers_translator[np.nan] = np.nan
customers_translator['.'] = np.nan

open_translator[np.nan] = np.nan
open_translator['.'] = np.nan

promo_translator[np.nan] = np.nan
promo_translator['.'] = np.nan

stateholiday_translator[np.nan] = np.nan
stateholiday_translator['.'] = np.nan

schoolholiday_translator[np.nan] = np.nan
schoolholiday_translator['.'] = np.nan

In [19]:
df['Sales'] = df['Sales'].apply(lambda x: sales_translator[x])
pd.isna(df).value_counts()

Store  DayOfWeek  Date   Sales  Customers  Open   Promo  StateHoliday  SchoolHoliday
False  False      False  False  False      False  False  False         False            25112
                  True   False  False      False  False  False         False            13783
                  False  False  False      False  False  True          False            13059
                                                  True   False         False             8050
                                                  False  False         True              8008
                                                                                        ...  
True   True       False  False  False      False  False  False         True               284
False  True       False  False  True       False  True   False         False              271
                                                  False  False         True               247
                                           True   False  False       